In [1]:
import os
import numpy as np
import zarr
from ome_zarr.io import parse_url
from ome_zarr.reader import Reader
import dask.array as da

from cellpose import models

import napari
from napari.settings import get_settings
get_settings().application.ipy_interactive = True

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

In [2]:
# pathway to the input data
input_path_zarr = r'D:\kasia\Yian_test_data\E2_ch00.zarr'

# read in the data
reader = Reader(parse_url(input_path_zarr))

image_node = list(reader())[0]
dask_data = image_node.data[0]
dask_data

c:\Users\kmk280\AppData\Local\miniconda3\envs\cellpose2-env\lib\site-packages\zarr\creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)


dask.array<from-zarr, shape=(481, 7557, 7557), dtype=uint16, chunksize=(10, 512, 512), chunktype=numpy.ndarray>

In [12]:
# specify if all frame is to be segmented (start 0 and stop -1 for the full movie)
params = {
    'frame_start': -10,
    'row_start': 3000,
    'col_start': 3000,
    'frame_stop': -1,
    'row_stop': 4000,
    'col_stop': 4000
}

# check that the values are acceptable

# modify the values if they are negative
data_shapes = [dask_data.shape[0], dask_data.shape[1], dask_data.shape[2]]
params = {key: value + data_shapes[i % 3] if value < 0 else value for i, (key, value) in enumerate(params.items())}
frame_start, row_start, col_start, frame_stop, row_stop, col_stop = [params[key] for key in params]


assert frame_start < frame_stop, f'frame_start {frame_start} should be smaller than frame_stop {frame_stop}'
assert row_start < row_stop, f'row_start {row_start} should be smaller than row_stop {row_stop}'
assert col_start < col_stop, f'col_start {col_start} should be smaller than col_stop {col_stop}'

In [13]:
# read cellpose model
model_name = 'cyto2'
model = models.CellposeModel(gpu=True,model_type = model_name)

### Segment to png files

In [14]:
masks_all = []
for i in tqdm(range(frame_start,frame_stop)):

    im_frame = dask_data[i,row_start:row_stop,col_start:col_stop]
    mask,_,_ = model.eval(im_frame,diameter = 60)

    masks_all.append(mask)

masks_all = np.array(masks_all)

  0%|          | 0/9 [00:00<?, ?it/s]

### Display to check segmentation quality

In [15]:
viewer = napari.Viewer()
viewer.add_image(image_node.data, name=model_name)
viewer.add_labels(masks_all, name='mask', translate=(frame_start,row_start,col_start))
viewer.dims.current_step = (frame_start,0,0)